# Model

In [64]:
# Install the Google Generative AI package (google-generativeai).
# This package is necessary to interact with Google's Gemini AI models.
!pip install google-generativeai  # Install the correct package for Google Gemini

# Install the LangChain package, which is a framework for building applications powered by language models.
# LangChain can be used to manage chains of prompts and calls to LLMs (Large Language Models) like Gemini.
!pip install langchain  # Install LangChain, a popular package for LLM workflows


In [65]:
# Import the 'os' module, which provides functions for interacting with the operating system
import os

# Import the 'generativeai' library from Google's Generative AI SDK
import google.generativeai as genai

# Configure the Generative AI API by providing the API key for authentication
genai.configure(api_key="API_Key")

# Create a configuration for the model generation
generation_config = {
  # The temperature controls the randomness of the output.
  # A higher value (like 1) results in more random responses.
  "temperature": 1,

  # top_p controls nucleus sampling, where the model considers only the most probable tokens
  # whose probabilities add up to top_p. A higher value includes more diverse tokens.
  "top_p": 0.95,

  # top_k specifies the number of highest probability tokens to be considered for sampling.
  # A value of 64 means the model will consider the top 64 tokens at each generation step.
  "top_k": 64,

  # max_output_tokens defines the maximum number of tokens the model can generate.
  # Setting it to 8192 allows for long responses.
  "max_output_tokens": 8192,

  # The MIME type for the response. In this case, we're requesting a plain text response.
  "response_mime_type": "text/plain",
}

# Create an instance of the generative model using the specified model name and configuration
model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",  # The specific model version being used
  generation_config=generation_config,  # The generation configuration defined above
)


# Output Parser


1. Getting **Structered info** than simple text
2. Classes
3. Parse : method taking as string(response from an LLM) & parse it into some structures

    ## Two Methods (output from str -> json)

1.  Pydantic --  fastest data validation
2.  ResponseSchema and StructeredOutputParser from Langchain





In [59]:
# Import the PydanticOutputParser from the LangChain Core module
# This parser is used to validate and parse model outputs based on Pydantic models
from langchain_core.output_parsers import PydanticOutputParser

# Importing required components from Pydantic for model validation
# BaseModel is the foundational class for creating Pydantic models, Field allows setting field attributes and descriptions, and validator enables validation logic for fields
from langchain_core.pydantic_v1 import BaseModel, Field, validator

# Creating a Pydantic class to define the structure of the expected output
class Person(BaseModel):  # Defines a Pydantic model named 'Person' to model data with specific fields
    # Defines a string field 'name' with a description for better clarity
    name: str = Field(description="The person's name")

    # Defines an integer field 'age' with a description
    age: int = Field(description="The person's age")

    # Defines a field 'zip_code' that can either be a string or None (optional)
    # The "|" symbol signifies a union type (str | None), allowing it to hold one of two types
    zip_code: str | None = Field(description="Zip/postal code")

    # Defines a required string field 'country'
    country: str  # This field does not have an explicit description, but it is required.

# Creates an instance of PydanticOutputParser with the 'Person' Pydantic model
# This parser will validate and parse the output to match the 'Person' schema
json_parser = PydanticOutputParser(pydantic_object=Person)



# Prompt Templates

In [60]:
# Import the PromptTemplate class from LangChain Core's prompts module
# PromptTemplate helps manage and format prompts for large language models (LLMs)
from langchain_core.prompts import PromptTemplate

# Define a prompt template with placeholders for dynamic content insertion
# The {topic} and {format_instructions} placeholders will be replaced by values when the template is formatted
template = """
Answer the following questions about {topic} with the format:

{format_instructions}

"""

# Create an instance of PromptTemplate
prompt_template = PromptTemplate(
    # The input_variables defines the list of dynamic variables that will be replaced in the template.
    # Here, "topic" is the variable that will be dynamically inserted into the prompt.
    input_variables=["topic"],

    # Pass the prompt template defined above
    template=template,

    # partial_variables allows you to specify static values for some variables.
    # "format_instructions" is provided with a static value using json_parser's format instructions.
    partial_variables={"format_instructions": json_parser.get_format_instructions()}
)

# Example usage of the prompt template with a specific topic
# Format the template by providing a value for the "topic" variable
prompt = prompt_template.format(topic="human")

# Print the formatted prompt to see how it looks
print(prompt)



Answer the following questions about human with the format:

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "The person's name", "type": "string"}, "age": {"title": "Age", "description": "The person's age", "type": "integer"}, "zip_code": {"title": "Zip Code", "description": "Zip/postal code", "type": "string"}, "country": {"title": "Country", "type": "string"}}, "required": ["name", "age", "country"]}
```




# Parsing into Json Output

In [61]:
# Generate content using the model by passing in the formatted prompt
response = model.generate_content(prompt)

# Print the response text generated by the model
print(response.text)

# The 'response.text' is expected to be a string, and the type can be verified by uncommenting the next line
# print(type(response.text))  # This would print <class 'str'>

# The response text can be parsed into a dictionary or Pydantic model using the json_parser's parse method.
# This converts the text response into a structured format defined by the Pydantic model (Person in this case).
output_dict = json_parser.parse(response.text)

# Print the parsed output as a Python dictionary or Pydantic model object.
# This object will contain the values extracted from the response text based on the Person schema.
print(output_dict)

# Print the type of output_dict to verify that it's now a structured object (e.g., a Pydantic model).
print(type(output_dict))  # Expected type: <class 'Person'> or <class 'dict'>

```json
{
"name": "John Doe",
"age": 30,
"zip_code": "12345",
"country": "USA"
}
```
name='John Doe' age=30 zip_code='12345' country='USA'
<class '__main__.Person'>
